In [1]:
!pip install flask flask-cors pyngrok tensorflow pillow


In [2]:
!ngrok authtoken 347CnKnBIAtcJ4q8P5lh25ssb1h_5D8VbG5BivuFKSVKYrGPY


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [3]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import torch
from torchvision import transforms as T
from PIL import Image
import timm
import io
from pyngrok import ngrok
from threading import Thread

# --- Flask setup ---
app = Flask(__name__)
CORS(app)

# --- Correct class names (must match training order!) ---
class_names = [
    "plastic waste",
    "metal waste",
    "paper waste",
    "battery waste",
    "glass waste",
    "automobile wastes",
    "organic waste",
    "E-waste",
    "light bulbs"
]

# --- Bin + suggestion mapping ---
bin_suggestions = {

    "plastic waste": {
        "bin": "Recycling (Plastic)",
        "suggestion": "Rinse plastic containers before recycling."
    },

    "metal waste": {
        "bin": "Recycling (Metal)",
        "suggestion": "Rinse cans and remove sharp edges."
    },

    "paper waste": {
        "bin": "Recycling (Paper)",
        "suggestion": "Recycle only clean, dry paper. No food stains."
    },

    "battery waste": {
        "bin": "Battery / E-Waste Recycling",
        "suggestion": "Do NOT throw in trash. Take batteries to an e-waste drop-off."
    },

    "glass waste": {
        "bin": "Recycling (Glass)",
        "suggestion": "Recycle clean NON-broken glass. Keep colors separate if required."
    },

    "automobile wastes": {
        "bin": "Special Vehicle Waste",
        "suggestion": "Car parts must be taken to an auto-recycling facility."
    },

    "organic waste": {
        "bin": "Compost",
        "suggestion": "Food scraps and plant waste go to compost."
    },

    "E-waste": {
        "bin": "Electronic Waste",
        "suggestion": "Recycle electronics at certified e-waste centers."
    },

    "light bulbs": {
        "bin": "Hazardous Waste",
        "suggestion": "Do NOT throw bulbs in trash. Dispose at hazardous collection."
    }
}

# --- PyTorch transforms ---
transform = T.Compose([
    T.Resize((224, 224)),
    T.ToTensor(),
    T.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

# --- Load ReXNet model ---
model_name = "rexnet_150"
num_classes = len(class_names)

model = timm.create_model(
    model_name=model_name,
    pretrained=False,
    num_classes=num_classes
)

# Load weights
state = torch.load("wastemodel.pth", map_location="cpu")
model.load_state_dict(state)
model.eval()

@app.route("/")
def home():
    return "🔥 PyTorch Waste Classifier (9 classes) API is running!"

@app.route("/predict", methods=["POST"])
def predict():
    if "file" not in request.files:
        return jsonify({"error": "No file uploaded"}), 400

    file = request.files["file"]
    img = Image.open(io.BytesIO(file.read())).convert("RGB")

    img_tensor = transform(img).unsqueeze(0)

    with torch.no_grad():
        logits = model(img_tensor)
        probs = torch.softmax(logits, dim=1)
        confidence, predicted_idx = torch.max(probs, 1)
        predicted_idx = predicted_idx.item()
        confidence = confidence.item()

    predicted_class = class_names[predicted_idx]

    info = bin_suggestions.get(predicted_class, {
        "bin": "Unknown",
        "suggestion": "No disposal info."
    })

    return jsonify({
        'predicted_class': predicted_class,
        'confidence': confidence,
        'bin': info['bin'],
        'suggestion': info['suggestion']
    })

# --- Run server ---
def run():
    app.run(port=5005)

Thread(target=run).start()

public_url = ngrok.connect(5005)
print("🚀 PUBLIC URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5005
INFO:werkzeug:Press CTRL+C to quit


🚀 PUBLIC URL: NgrokTunnel: "https://hypernutritive-marley-untheoretically.ngrok-free.dev" -> "http://localhost:5005"
